# Setting Up

#### Get Kaggle API, load the dataset, and correct the previous implementations of SA2C.py

## Get the SA2C.py Code

In [3]:
%%bash --out output --err error
pip install pandas trfl kaggle
unzip SA2C_code.zip

### Download the dataset

In [1]:
%%bash --out output --err error
mkdir ~/.kaggle
cp /content/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

kaggle datasets download -d retailrocket/ecommerce-dataset
mkdir -p "ecommerce-dataset"
unzip ecommerce-dataset.zip -d "ecommerce-dataset"

In [4]:
%cd /content/SA2C_code/Kaggle

/content/SA2C_code/Kaggle


In [ ]:
%%bash --out output --err error
tf_upgrade_v2 \
  --infile 'SA2C.py' \
  --outfile 'SA2C_new.py' \
  --reportfile report_SA2C.txt

In [ ]:
%%bash --out output --err error
tf_upgrade_v2 \
  --infile 'SNQN.py' \
  --outfile 'SNQN_new.py' \
  --reportfile report_SNQN.txt

In [ ]:
%%bash --out output --err error
tf_upgrade_v2 \
  --infile 'SA2C_Features.py' \
  --outfile 'SA2C_Features_new.py' \
  --reportfile report_SA2C_features.txt

In [5]:
%%bash --out output --err error
tf_upgrade_v2 \
  --infile 'SNQN_Features.py' \
  --outfile 'SNQN_Features_new.py' \
  --reportfile SNQN_features.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, GRU, Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import pandas as pd
import numpy as np


In [ ]:
PATH = '/content/ecommerce-dataset'
df_category=pd.read_csv(f"{PATH}/category_tree.csv")
df_events=pd.read_csv(f"{PATH}/events.csv")
df_sorted_events = pd.read_pickle('/content/SA2C_code/Kaggle/data/sorted_events.df')
df_items1=pd.read_csv(f"{PATH}/item_properties_part1.csv")
df_items2=pd.read_csv(f"{PATH}/item_properties_part2.csv")

# Combining Item Properties



In [ ]:
#item ids in the events.csv
events_item_ids = df_sorted_events.item_id.unique()
print(events_item_ids.shape)

# combine df_items
df_items = pd.concat([df_items1, df_items2], axis=0)
print(df_items.shape)

(70852,)
(20275902, 4)


In [ ]:
# get item for events items
item_features_df=df_items[(df_items.itemid.isin(events_item_ids)) &
            (df_items['property']=='categoryid')].drop(['property', 'timestamp'], axis = 1).drop_duplicates()

# drop missing values
missing_items = pd.DataFrame(set(events_item_ids) - set(item_features_df.itemid.unique()), columns = ['itemid'])
missing_items['value'] = np.nan

# concat missing items
item_features_df = pd.concat([item_features_df, missing_items], axis=0)
item_features_df.columns = ['itemid', 'categoryid']
item_features_df.categoryid = item_features_df.categoryid.astype(float)

item_category_features = item_features_df.merge(df_category, on='categoryid', how = 'left').drop_duplicates()
parentsids = item_category_features.parentid.unique()

In [ ]:
new_data = []
for item in events_item_ids:
  row = item_category_features[item_category_features.itemid == item]
  if len(row) == 0:
    categorical = parent = np.nan
  else:
    categorical = row.categoryid.tolist()
    parent = row.parentid.tolist()

  new_data.append({'itemid': item, 'categoryid': categorical, 'parentid':parent})

item_category_features_df = pd.DataFrame(new_data)

In [ ]:
one_hot_encoded = []
for idx in range(item_category_features_df.shape[0]):
  one_hot_encoded.append(np.isin(parentsids,
                                 item_category_features_df.parentid[idx]).astype(int))

one_hot_encoded = np.array(one_hot_encoded)
one_hot_encoded = pd.DataFrame(one_hot_encoded, index = item_category_features_df.itemid,
             columns = parentsids)

In [ ]:
one_hot_encoded.to_csv('/content/SA2C_code/Kaggle/data/item_features.csv')

# Comparing Different Product Recommendation Recommenders

## SA2C

### GRU Model


| Metric                                    | 4200                    | 8200                    | 12200                   | 16200                   |
|-------------------------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Batch                                     | 4200                    | 8200                    | 12200                   | 16200                   |
| Cumulative Reward @ 5                      | 5947.0                  | 8100.8                  | 8759.8                  | 8293.4                  |
| Clicks HR NDCG @ 5                        | 0.169476                | 0.23468                 | 0.254839                | 0.241974                |
| Purchase HR NDCG @ 5                      | 0.366093                | 0.481572                | 0.515971                | 0.485352                |
| Cumulative Reward @ 10                     | 6879.6                  | 9317.6                  | 10122.6                 | 9642.6                  |
| Clicks HR NDCG @ 10                       | 0.200776                | 0.276005                | 0.302757                | 0.288726                |
| Purchase HR NDCG @ 10                     | 0.402381                | 0.526744                | 0.559252                | 0.53128                 |
| Cumulative Reward @ 15                     | 7426.0                  | 9965.8                  | 10883.0                 | 10412.2                 |
| Clicks HR NDCG @ 15                       | 0.218839                | 0.299469                | 0.329273                | 0.314912                |
| Purchase HR NDCG @ 15                     | 0.424872                | 0.544321                | 0.584389                | 0.55963                 |
| Cumulative Reward @ 20                     | 7823.6                  | 10440.0                 | 11383.8                 | 10929.4                 |
| Clicks HR NDCG @ 20                       | 0.231755                | 0.316172                | 0.346677                | 0.332925                |
| Purchase HR NDCG @ 20                     | 0.44226                 | 0.559252                | 0.60121                 | 0.576829                |
| Off-line Corrected Evaluation @ 10         | 0.029068, 0.084085      | 0.056642, 0.151099      | 0.07279, 0.180657       | 0.062292, 0.156229      |


In [ ]:
!python SA2C_new.py --model=GRU --epoch=5

2023-11-16 23:29:54.140264: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 23:29:54.140309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 23:29:54.140341: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 23:29:55.364575: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/SA2C_code/Kaggle/SA2C_new.py:88: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.


### Caesar Model


| Metric                                    | 4200                    | 8200                    | 12200                   | 16200                   |
|-------------------------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Batch                                     | 4200                    | 8200                    | 12200                   | 16200                   |
| Cumulative Reward @ 5                      | 3168.6                  | 5102.0                  | 5988.8                  | 6165.2                  |
| Clicks HR NDCG @ 5                        | 0.092709                | 0.1783                  | 0.183811                | 0.183811                |
| Purchase HR NDCG @ 5                      | 0.184275                | 0.33453                 | 0.343224                | 0.343224                |
| Cumulative Reward @ 10                     | 3658.0                  | 5886.2                  | 7103.2                  | 7663.4                  |
| Clicks HR NDCG @ 10                       | 0.108955                | 0.176796                | 0.214875                | 0.233479                |
| Purchase HR NDCG @ 10                     | 0.20412                 | 0.321867                | 0.381591                | 0.404271                |
| Cumulative Reward @ 15                     | 3979.0                  | 6342.2                  | 7663.4                  | 8040.0                  |
| Clicks HR NDCG @ 15                       | 0.119436                | 0.191546                | 0.233479                | 0.246353                |
| Purchase HR NDCG @ 15                     | 0.217917                | 0.34209                 | 0.404271                | 0.417879                |
| Cumulative Reward @ 20                     | 4216.8                  | 6670.0                  | 8040.0                  | 8040.0                  |
| Clicks HR NDCG @ 20                       | 0.127627                | 0.202483                | 0.246353                | 0.246353                |
| Purchase HR NDCG @ 20                     | 0.226233                | 0.355131                | 0.417879                | 0.417879                |
| Off-line Corrected Evaluation @ 10         | 0.009152, 0.022983      | 0.023722, 0.059393      | 0.034693, 0.084227      | 0.037460, 0.088465      |



In [ ]:
! python SA2C_new.py --model="Caser" --epoch=5

2023-11-17 01:34:28.565836: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 01:34:28.565891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 01:34:28.565921: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 01:34:29.640996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/SA2C_code/Kaggle/SA2C_new.py:151: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  self.states_hidden = tf.compat.v1.layers.dropout(self.final

## SNQN

| Batch | 200   | 4200    | 8200    | 12200   | 16200   |
|-------|-------|---------|---------|---------|---------|
| Cumulative Reward @ 5 | 2.4   | 5913.0  | 8012.8  | 8774.4  | 9006.0  |
| Clicks HR NDCG @ 5    | 0.000101 | 0.169518 | 0.232397 | 0.255034 | 0.262962 |
| Purchase HR NDCG @ 5  | 0.000000 | 0.359478 | 0.475146 | 0.517861 | 0.526177 |
| Cumulative Reward @ 10 | 3.0   | 6824.2  | 9223.2  | 10139.6 | 10439.8 |
| Clicks HR NDCG @ 10   | 0.000127 | 0.200125 | 0.274466 | 0.302123 | 0.311472 |
| Purchase HR NDCG @ 10 | 0.000000 | 0.394821 | 0.515782 | 0.565300 | 0.580231 |
| Cumulative Reward @ 15 | 4.2   | 7371.6  | 9885.0  | 10874.8 | 11221.8 |
| Clicks HR NDCG @ 15   | 0.000178 | 0.218442 | 0.297956 | 0.327870 | 0.338985 |
| Purchase HR NDCG @ 15 | 0.000000 | 0.416367 | 0.535816 | 0.589114 | 0.604990 |
| Cumulative Reward @ 20 | 7.0   | 7748.8  | 10361.0 | 11346.2 | 11729.6 |
| Clicks HR NDCG @ 20   | 0.000254 | 0.230960 | 0.314608 | 0.345088 | 0.357319 |
| Purchase HR NDCG @ 20 | 0.000189 | 0.431676 | 0.551314 | 0.601210 | 0.618976 |


In [ ]:
!python SNQN_new.py --model=GRU --epoch=5

2023-11-24 19:56:22.026405: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 19:56:22.026456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 19:56:22.026482: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 19:56:22.033955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:56:23.070951: W tensorflow/compiler/

#### Item Features

| Batch   | 200    | 4200     | 8200     | 12200    | 16200    |
|---------|--------|----------|----------|----------|----------|
| Cumulative Reward @ 5                     | 1.248   | 5894.19  | 8245.272 | 9002.632 | 9306.576 |
| Clicks HR NDCG @ 5                       | 8.32e-06 | 0.167892 | 0.237364 | 0.261763 | 0.272984 |
| Purchase HR NDCG @ 5                     | 2.0116e-04 | 0.359872 | 0.496682 | 0.533072 | 0.545013 |
| Cumulative Reward @ 10                    | 2.08    | 6785.92  | 9453.216 | 10369.84 | 10794.16 |
| Clicks HR NDCG @ 10                      | 4.368e-05 | 0.197216 | 0.278689 | 0.309344 | 0.323616 |
| Purchase HR NDCG @ 10                    | 2.0116e-04 | 0.3992   | 0.54056  | 0.582992 | 0.594048 |
| Cumulative Reward @ 15                    | 5.432   | 7322.368 | 10156.608| 11123.84 | 11570.568|
| Clicks HR NDCG @ 15                      | 1.512e-04 | 0.215472 | 0.302416 | 0.33364  | 0.348856 |
| Purchase HR NDCG @ 15                    | 4.0224e-04 | 0.419904 | 0.565536 | 0.601488 | 0.62224  |
| Cumulative Reward @ 20                    | 8.112   | 7681.52  | 10554.816| 11590.248| 12046.528|
| Clicks HR NDCG @ 20                      | 2.576e-04 | 0.227808 | 0.317016 | 0.350416 | 0.366144 |
| Purchase HR NDCG @ 20                    | 4.0224e-04 | 0.432192 | 0.576192 | 0.609952 | 0.623904 |
| Off-line Corrected Evaluation (Click_NG, Purchase_NG) @ 10 | - | 0.024801, 0.061776 | 0.036089, 0.088416 | 0.0388, 0.09204 | 0.041664, 0.097032 |



In [ ]:
! python SNQN_Features_new.py --model=GRU --epoch=5

2023-11-25 02:28:56.510605: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 02:28:56.510656: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 02:28:56.510694: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 02:28:56.518709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 02:28:57.828588: W tensorflow/c

## Observations from the Results
The comparison between the two models reveals distinct patterns. Key metrics, including cumulative rewards, hit rate (hr), and normalized discounted cumulative gain (ndcg) at various top-k recommendations (5, 10, 15, 20), highlight the impact of incorporating item features. This improvement is evident when contrasting outputs from SNQN_new.py and SAC2_new.py (without item features) with SNQN_Features_new.py (with item features). Notably, across both models, performance metrics show enhancement with an increasing number of top-k recommendations, aligning with typical trends in recommender systems. The decline in loss over batches for both models indicates ongoing learning and optimization. Interestingly, enough we find that SNQN model performs much better than SAC2 model.

The model featuring item features seems to perform comparitively to its counterpart, maybe if we provided better item features we would see that item features elevate the recommendation quality. This effect is particularly pronounced for cold items/users.


## Conclusion
The approach and results align seamlessly with the specified requirements. The successful training and comparison of session-based DRL recommenders, with and without item features, underscore the effectiveness of including such features in enhancing recommendation performance, especially in the context of e-commerce applications.